In [2]:
from pandas_summary import DataFrameSummary
from pathlib import Path

import os, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set()
os.chdir('/home/krivas/projects/analysis-project/')
from src.utils import convertInt, convertDate, add_datediffs, add_dayscount

%matplotlib inline

In [ ]:
DATA        = Path('data')
RAW         = DATA/'raw'
PROCESSED   = DATA/'processed'

# Transforming data

In [4]:
csv = pd.read_excel(RAW/'HistoricoJob01.csv.xlsx')

In [ ]:
%%time
for count in range(2, 8):
    csv = pd.concat([csv, pd.read_excel(RAW/f'HistoricoJob0{count}.csv.xlsx')])

In [ ]:
csv['Duracion'] = csv['Duracion'].astype(str)
csv['Promedio'] = csv['Promedio'].astype(str)
csv['Fecha_Ejec_Inicio'] = csv['Fecha_Ejec_Inicio'].astype(str)
csv['Fecha_Ejec_Fin'] = csv['Fecha_Ejec_Fin'].astype(str)
csv['Fecha_Carga_Scheduler'] = csv['Fecha_Carga_Scheduler'].astype(str)

In [ ]:
%%time
csv['duracion_int'] = csv.apply(lambda row: convertInt(row, 'Duracion'), axis=1) 

In [ ]:
%%time
csv['promedio_int'] = csv.apply(lambda row: convertInt(row, 'Promedio'), axis=1) 

In [ ]:
%%time
csv[['Fecha_Ejec_Inicio_Int', 'Hora_Ejec_Inicio_Int']] = csv.apply(lambda row: convertDate(row, 'Fecha_Ejec_Inicio'), axis=1)

In [ ]:
%%time
csv[['Fecha_Ejec_Fin_Int', 'Hora_Ejec_Fin_Int']] = csv.apply(lambda row: convertDate(row, 'Fecha_Ejec_Fin'), axis=1)

In [ ]:
cat_columns = ['Agente', 'Mxrc', 'Maxcmpc', 'Grupo', 'Force_Complete', 'CCF', 'Estado']
id_columns = ['Id_Job', 'Id_Malla']
for column in cat_columns:
    csv[column] = csv[column].astype('category').cat.codes

In [ ]:
csv['hubo_error'] = csv.Mxrc.apply(lambda row: 0 if row == 0 else 1)

In [ ]:
EARLY_MORNING = 21600
MORNING = 43200      
AFTERNOON = 64800

def decide_time(vals):

    if vals < EARLY_MORNING:
        return 0
    elif vals < MORNING:
        return 1
    elif vals < AFTERNOON:
        return 2
    else:
        return 3

csv['tiempo'] = csv.apply(lambda row: decide_time(row['Hora_Ejec_Inicio_Int']), axis=1)

In [ ]:
csv.to_csv(PROCESSED/'HistoricoJob.csv', index=False)